In [26]:
import httpx
import json
from typing import Dict
from urllib.parse import quote


In [27]:
import asyncio
from playwright.async_api import async_playwright
import re
import json

from dotenv import load_dotenv
#load the .env file
# get USERNAME and PASSWORD
load_dotenv(override=True)
import os
username = os.getenv("USERNAME")
password = os.getenv("PASSWORD")

In [28]:
#signon doesn't work rn

import asyncio
from playwright.async_api import TimeoutError

async def signon(page, username, password):
    try:
        await page.goto("https://www.instagram.com/accounts/login/", wait_until="networkidle")

        await page.wait_for_selector('input[name="username"]', timeout=10000)
        print("Filling in login form...")
        await page.fill('input[name="username"]', username)
        await page.fill('input[name="password"]', password)
        #sleep 500ms
        await page.wait_for_timeout(500)
        print("Hitting login button")
        
        # Wait for the login button to be visible (both selectors)
        # await page.getByRole('button', { aria-label=="Log in" }).click();
        await page.locator('[role="button"][aria-label-id="replay"]').click();


        # wait for networkidle
        await page.wait_for_timeout(5000)
        
        
        # print("Networkidle #1")
        # wait for things to settle
        await page.wait_for_timeout(1000)
        # Check for any popups or modals

        # Verify successful login by checking URL or user-specific elements
        if "login" in page.url:
            raise ValueError("Login failed: Incorrect username or password.")

        print("✅ Successfully logged in")
        
        return

    except TimeoutError:
        raise TimeoutError("Login timed out. Check credentials or network connection.")
    
    

In [29]:
import asyncio
from playwright.async_api import TimeoutError

async def get_total_posts(page):
    """Extracts the total number of posts from the user's profile."""
    total_posts = await page.evaluate("""
        () => {
            const element = document.querySelector('header section ul li span');
            return element ? parseInt(element.innerText.replace(',', '')) : null;
        }
    """)
    return total_posts if total_posts is not None else float('inf')

async def scrape_instagram_posts(userhandle: str, max_posts: int):
    """
    Scrapes Instagram post metadata for regular posts (excluding reels) from a given user's profile.
    """
    global context, page  # You must define these in the notebook before calling this function.

    # Step 1: Navigate to the user profile.
    profile_url = f"https://www.instagram.com/{userhandle}/"
    await page.goto(profile_url)
    await page.wait_for_load_state("networkidle")
    
    total_posts = await get_total_posts(page)
    scrape_limit = min(max_posts, total_posts)
    print(f"🔎 Total posts on profile: {total_posts}. Will scrape up to: {scrape_limit} posts.")

    # Step 2: Scroll and collect links as we go
    print("🔽 Scrolling and collecting post links...")
    unique_posts = {}
    scroll_attempts = 0
    MAX_SCROLL_ATTEMPTS = 20

    while len(unique_posts) < scrape_limit and scroll_attempts < MAX_SCROLL_ATTEMPTS:
        candidate_elements = await page.query_selector_all("a:has(div._aagu)")
        for element in candidate_elements:
            href = await element.get_attribute("href")
            if href and "/p/" in href and href not in unique_posts:
                unique_posts[href] = None
        print(f"🧠 Collected so far: {len(unique_posts)} / {scrape_limit}")
        if len(unique_posts) >= scrape_limit:
            break
        await page.evaluate("window.scrollTo(0, document.body.scrollHeight);")
        await asyncio.sleep(1.5)
        scroll_attempts += 1

    post_hrefs = list(unique_posts.keys())[:scrape_limit]
    results = {}

    # Step 3: Process each post in a new tab and intercept metadata
    print(f"📝 Opening and intercepting {len(post_hrefs)} posts...")
    for i, href in enumerate(post_hrefs, start=1):
        post_url = f"https://www.instagram.com{href}"
        new_page = await context.new_page()
        try:
            async with new_page.expect_response(
                lambda response: "/api/v1/media/" in response.url and "/info/" in response.url,
                timeout=5000
            ) as response_info:
                await new_page.goto(post_url)

            response = await response_info.value
            data = await response.json()
            results[href] = data
            print(f"✅ ({i}/{len(post_hrefs)}) Scraped metadata for {href}")
        except TimeoutError:
            print(f"⏱️ ({i}/{len(post_hrefs)}) Timeout: No metadata for {href}")
        finally:
            await new_page.close()

    return results


# Initiation

In [30]:
#This is a dynamic environment that stays open after execution

# Global references for easy access across notebook cells
playwright = await async_playwright().start()
device = playwright.devices["Pixel 5"]
browser = await playwright.chromium.launch(headless=False)  # Browser will stay open
context = await browser.new_context(**device)
page = await context.new_page()
# You now have persistent global references:
# playwright, browser, context, page

# load cookies only if they exist
if os.path.exists("cookies.json"):
    print("Loading cookies from cookies.json")
    # load cookies
    with open("cookies.json", "r") as f:
        cookies = json.load(f)

    # set cookies
    await page.context.add_cookies(cookies)
else:
    print("No cookies found, logging in...")
    # login
    await signon(page, username, password)
    # get cookies
    cookies = await page.context.cookies()
    # save cookies
    with open("cookies.json", "w") as f:
        json.dump(cookies, f)

FileNotFoundError: [Errno 2] No such file or directory: 'ig_session.json'

In [16]:
# save browser cookies to a file
cookies = await context.cookies()
with open('cookies.json', 'w') as f:
    json.dump(cookies, f)
# Close the browser when done

In [21]:
results = await scrape_instagram_posts("mattrife", 50)
print(json.dumps(results, indent=2))

Total posts on profile: 1554. Will scrape: 50 posts.
🔽 Scrolling to load more posts...
